In [25]:
import xlsxwriter
import json
import os
from util import json_write, json_read

In [26]:
def button_options(width=90, height=35, x_offset=10, y_offset=10, url='internal:HOME!A1', font_size=11):
    return {
        'width': width, 'height': height,
        'x_offset': x_offset, 'y_offset': y_offset,
        'font': {'name': 'Times New Roman', 'color': 'white', 'size': font_size, 'bold': True},
        'align': {'vertical': 'middle', 'horizontal': 'center'},
        'gradient': {'colors': ['#00B050', '#00B050']},
        'line': {'color': '#00B050', 'width': 0},
        'url': url,
    }

In [27]:
def get_sheet_data(word_list):
    sheet_data = []
    for word in word_list:
        word_index = word['index']
        vn_sound = word['vn_sound']
        kanji = word['word']
        strokes_image_path = ''

        onyomi_example = word['onyomi_example']
        onyomi_example = onyomi_example.split('###')
        onyomi_example = [f'** {text}' for text in onyomi_example]
        onyomi_example = '\n'.join(onyomi_example)

        kunyomi_example = word['kunyomi_example']
        kunyomi_example = kunyomi_example.split('###')
        kunyomi_example = [f'** {text}' for text in kunyomi_example]
        kunyomi_example = '\n'.join(kunyomi_example)

        japanese_char = "** 訓(Onyomi)" + onyomi_example.strip() + '\n={}\n'.format('='*28) + '** 訓(Kunyomi)' + kunyomi_example.strip()
        japanese_char = japanese_char.replace('** 訓(Onyomi)**', '** 訓(Onyomi):')
        japanese_char = japanese_char.replace('** 訓(Kunyomi)**', '** 訓(Kunyomi):')

        content1 = word['components'] + word['comments']
        content1 = "** Thành phần: " + word['components'] + '\n={}\n'.format('='*38) + '** Comments: ' + word['comments']

        content2 = word['related_kanjis']

        search_url = [word['kanji_mazzi_url'], word['kanji_jdict_url']]

        sheet_data.append([word_index, vn_sound, kanji, strokes_image_path, japanese_char, content1, content2, search_url])
    return sheet_data

In [28]:
ZOOM_PERCENT_SHEET_NORMAL = 95
ZOOM_PERCENT_SHEET_ALL = 50

database_folder = 'Kanji Database'
output_folder = 'Output'
stroke_image_folder = 'Stroke Image'
stroke_image_path = os.path.join(database_folder, stroke_image_folder)

n5_db_fname = 'N5_database.json'
n4_db_fname = 'N4_database.json'
course_db_fname = 'Course_database.json'

base_out_fname = 'ByVing'

courses = ['Course'] 
courses = ['N5']
courses = ['N5', 'N4', 'Course']

print(course_db_fname.split('_')[1].replace('.json', ''))

for course in courses:
    print("\n>> Course: ", course)
    base_fname = course_db_fname.split('_')[1].replace('.json', '')
    input_file_name = f'{course}_{base_fname}.json'

    input_file_path = os.path.join(database_folder, input_file_name)
    input_data = json_read(input_file_path)

    output_file_name = f'{course} Kanji {base_out_fname}.xlsx'
    output_file_path = os.path.join(output_folder, output_file_name)
    try: 
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
    except Exception as bug:
        print(f'Create folder {output_folder} EROOR', bug)
        break
    
    print(f">> Open: [ {output_file_path} ]")
    workbook = xlsxwriter.Workbook(output_file_path)   

    # ============================================================
    home_sheet_name = 'HOME'
    all_sheet_name = 'ALL'

    # ================== SHEET NAME: HOME =========================
    print(f">> Create sheet: {home_sheet_name}")
    worksheet = workbook.add_worksheet(home_sheet_name) 
    worksheet.hide_gridlines(2) # Hide_gridlines Options: 0) Don’t hide gridlines. 1)Hide printed gridlines only. 2) Hide screen and printed gridlines
    
    total_lesson = len(input_data[course])
    total_row_home_control = 5
    total_col_home_control = int(total_lesson / total_row_home_control)

    # x_offset_home_control = 8 if course!='Course' else 8
    x_offset_home_control = 8
    y_offset_home_control = 9 if course!='Course' else 6
    
    first_lesson = 26 if course=='N4' else 1

    homesheet_btn_height = 49
    for row in range(x_offset_home_control, x_offset_home_control + total_row_home_control):
        worksheet.set_row(row, height=homesheet_btn_height)
        for col in range(y_offset_home_control, y_offset_home_control + total_col_home_control):
            if row == x_offset_home_control and col == y_offset_home_control:
                options = button_options(width=150, height=50, x_offset=0, y_offset=0, url=f'internal:{all_sheet_name}!A1', font_size=14)
                worksheet.insert_textbox(row-3, col+1, f"KANJI {all_sheet_name}", options)
            
            lesson_name = first_lesson + total_col_home_control * (row - x_offset_home_control) + (col - y_offset_home_control)
            options = button_options(height=homesheet_btn_height, x_offset=10, y_offset=8, url=f'internal:{lesson_name}!A1', font_size=14)
            worksheet.insert_textbox(row, col, f"Bài {lesson_name}", options)
            
            worksheet.set_column(row, col, 15)
            worksheet.write_string(row, col, '', workbook.add_format({'border': 2}))
    
    # ================== SHEET NAME: ALL =========================
    print(f">> Create sheet: {all_sheet_name}")
    worksheet = workbook.add_worksheet('ALL') 
    worksheet.set_zoom(ZOOM_PERCENT_SHEET_ALL) # zoom level: 30%
    worksheet.hide_gridlines(2) # Hide_gridlines Options: 0) Don’t hide gridlines. 1)Hide printed gridlines only. 2) Hide screen and printed gridlines

    end_sheet_index = total_lesson + 1
    button_middel_sheet_index = 7

    worksheet.set_row(0, height=150)

    home_sheet_options = button_options(width=400, height=150 ,x_offset=20, y_offset=20, url=f'internal:{home_sheet_name}!A1', font_size=40)
    worksheet.insert_textbox(0, button_middel_sheet_index, home_sheet_name, home_sheet_options)
    bottom_sheet_options = button_options(width=400, height=150 ,x_offset=20, y_offset=20, url=f'internal:{all_sheet_name}!A{end_sheet_index}', font_size=40)
    worksheet.insert_textbox(0, button_middel_sheet_index+2, f"CUỐI TRANG", bottom_sheet_options)

    worksheet.insert_textbox(end_sheet_index, button_middel_sheet_index, home_sheet_name, home_sheet_options)
    top_sheet_options = button_options(width=400, height=150 ,x_offset=20, y_offset=20, url=f'internal:{all_sheet_name}!A1', font_size=40)
    worksheet.insert_textbox(end_sheet_index, button_middel_sheet_index+2, f"ĐẦU TRANG", top_sheet_options)
        
    for lesson, word_list in input_data[course].items():
        row = (int(lesson)-26 if course=='N4' else int(lesson)-1) + 1
        total_columnn_all = len(word_list)
        worksheet.set_row(row, height=187.5)
        sheet_data = get_sheet_data(word_list)
        for col in range(total_columnn_all + 1):
            if col == 0:
                worksheet.set_column(row, col, width=15)
                worksheet.write_string(row, col, f'B{lesson}', workbook.add_format({
                        'font': 'Times New Roman', 'font_size': 40, 'bold': True, 'color': '#00B050',
                        'text_wrap': True, 'align': 'center', 'valign': 'vcenter', 'border': True
                    })
                )
            else:
                word = word_list[col-1]['word']
                file_name = f'{lesson}_{col}_{word}.png'
                strokes_image_path = os.path.join(database_folder, stroke_image_folder, lesson, file_name)

                worksheet.set_column(row, col, width=35)
                if col == total_columnn_all:
                    worksheet.write_string(row, col, '', workbook.add_format({'top': True, 'bottom': True, 'right': True}))
                else:
                    worksheet.write_string(row, col, '', workbook.add_format({'top': True, 'bottom': True}))

                # word_url = f'internal:{lesson}!A{col}' if col<4 else f'internal:{lesson}!A{col+2}'
                col_word = lambda col: chr(65 + col)
                word_url = f'internal:{lesson}!{col_word(3)}{col+1}'
                worksheet.insert_image(row, col, strokes_image_path, {
                    'x_scale': 1, 'y_scale': 1,
                    'url': word_url,
                })

    for lesson, word_list in input_data[course].items():
        title = ['🔢STT', '📝HÁN VIỆT', '🈴CHỮ HÁN', '✒📌NÉT VIẾT', '🎶ON & KUN', '📘THÀNH PHẦN', '📘TỪ VỰNG', '🔎TRA CỨU', '']
        sheet_data = get_sheet_data(word_list)
        for i in range(len(sheet_data)):
            for __ in range(len(title) - len(sheet_data[0])):
                sheet_data[i].append('')
        sheet_data.insert(0, title)

        # print('\r' + f">> Create sheet: {lesson}")
        worksheet = workbook.add_worksheet(f'{lesson}')
        worksheet.set_zoom(ZOOM_PERCENT_SHEET_NORMAL) # zoom level: 80%
        worksheet.hide_gridlines(2) # Hide_gridlines Options: 0) Don’t hide gridlines. 1)Hide printed gridlines only. 2) Hide screen and printed gridlines

        total_row = len(sheet_data)
        for row in range(total_row):
            word = sheet_data[row][2]
            total_column = len(sheet_data[row])
            if row == 0:
                worksheet.set_row(row, height=20)
                for col in range(len(sheet_data[row][2])):
                    worksheet.write_string(row, col, sheet_data[row][col], workbook.add_format({
                        'font': 'Cambria', 'font_size': 10, 'color': '#002060', 'bg_color': '#92D050', 'bold': True,
                        'text_wrap': True, 'align': 'center', 'valign': 'vcenter',
                        'border': True
                    }))
            else:
                worksheet.set_row(row, height=187.5)
                col_define = {
                    'stt': 0,
                    'vn_sound': 1,
                    'kanji': 2,
                    'stroke': 3,
                    'on_kun': 4,
                    'content': 5,
                    'similar': 6,
                    'search': 7
                }
                for col in range(len(sheet_data[row])):
                    item = sheet_data[row][col]
                    # ============== WORD INDEX =================
                    if col == col_define['stt']:
                        worksheet.set_column(row, col, width=7)
                        worksheet.write_number(row, col, int(item), workbook.add_format({
                                'font': 'Times New Roman', 'font_size': 10, 'bold': True,
                                'text_wrap': True, 'align': 'center', 'valign': 'vcenter',
                                'border': True
                            })
                        )

                    # ============== VN SOUND =================
                    elif col == col_define['vn_sound']:
                        worksheet.set_column(row, col, width=12)
                        worksheet.write_string(row, col, item, format)
                        worksheet.write_string(row, col, item, workbook.add_format({
                                'font': 'Times New Roman', 'font_size': 12, 'bold': True,
                                'text_wrap': True, 'align': 'center', 'valign': 'vcenter',
                                'border': True
                            })
                        )

                    # ============== KANJI =================
                    elif col == col_define['kanji']:
                        worksheet.set_column(row, col, width=12)
                        worksheet.write_string(row, col, item, workbook.add_format({
                                'font': 'MS Mincho', 'color': '#00B050', 'font_size': 36, 'bold': True,
                                'text_wrap': True, 'align': 'center', 'valign': 'vcenter',
                                'border': True
                            })
                        )

                    # ============== STROKES IMAGE =================
                    elif col == col_define['stroke']:
                        #----------------------------------------------------------
                        file_name = f'{lesson}_{row}_{word}.png'
                        strokes_image_path = os.path.join(database_folder, stroke_image_folder, lesson, file_name)

                        worksheet.set_column(row, col, width=35)
                        worksheet.write_string(row, col, '', workbook.add_format({'border': True}))

                        col_word = lambda col: chr(65 + col)
                        col_in_all = int(sheet_data[row][0])
                        worksheet.insert_image(row, col, strokes_image_path, {
                            'x_scale': 1, 'y_scale': 1,
                            # 'url': f'https://jdict.net/search?keyword={word}&type=kanji'
                            'url': f'internal:{all_sheet_name}!{col_word(col_in_all)}{int(lesson)+1}'
                        })
                    
                    # ============== ONYOMI AND KUNYOMI =================
                    elif col == col_define['on_kun']:
                        worksheet.set_column(row, col, width=45)
                        worksheet.write_string(row, col, item, workbook.add_format({
                                'font': 'Yu Gothic UI Semibold', 'font_size': 11,
                                'text_wrap': True, 'align': 'left', 'valign': 'vcenter',
                                'border': True
                            })
                        )

                    # ============== CONTENT =================
                    elif col == col_define['content']:
                        worksheet.set_column(row, col, width=60)
                        worksheet.write_string(row, col, item, workbook.add_format({
                                'font': 'Yu Gothic UI Semibold', 'font_size': 11,
                                'text_wrap': True, 'align': 'left', 'valign': 'top',
                                'border': True
                            })
                        )

                    elif col == col_define['similar']:
                        worksheet.set_column(row, col, width=60)
                        worksheet.write_string(row, col, item, workbook.add_format({
                                'font': 'Yu Gothic UI Semibold', 'font_size': 11,
                                'text_wrap': True, 'align': 'left', 'valign': 'top',
                                'border': True
                            })
                        )

                    # ============== MAZZI SEARCH URL =================
                    elif col == col_define['search']:
                        options = button_options(width=70, x_offset=10, y_offset=75, url=item[0])
                        worksheet.insert_textbox(row, col, f"Mazzi", options)

                        worksheet.set_column(row, col, width=12)
                        worksheet.write_string(row, col, '', workbook.add_format({'border': True}))

                        col_word = lambda col: chr(65 + col)
                        col_in_all = int(sheet_data[row][0])
                        options = button_options(width=70, x_offset=10, y_offset=130, url=item[1])
                        worksheet.insert_textbox(row, col, f"Jdcit", options)
                    
                    # ============== BUTTON ===========================
                    elif col == (len(title) - 1):
                        row_index = 1    
                        col_index = col 
                        with_btn = 120
                        height_btn_right = 35
                        x_offset_btn = 20
                        y_offset_btn = lambda index: index*(height_btn_right + 20) + 75
                        # ====================================================================
                        worksheet.set_column(row_index, col_index, width=20)
                        # ====================================================================
                        options = button_options(width=with_btn, x_offset=x_offset_btn, y_offset=y_offset_btn(0), url=f'internal:{home_sheet_name}!A1')
                        worksheet.insert_textbox(row_index, col_index, home_sheet_name, options)
                        # ====================================================================
                        col_word = lambda col: chr(65 + col)
                        col_index_goto = len(word_list)+1 if len(word_list) <= 12 else int(len(word_list) / 2)+1
                        options = button_options(width=with_btn, x_offset=x_offset_btn, y_offset=y_offset_btn(1), url=f'internal:{all_sheet_name}!{col_word(col_index_goto)}{int(lesson)+1}')
                        worksheet.insert_textbox(row_index, col_index, 'ALL TAB', options)
                        # ====================================================================
                        bottom_sheet_options = button_options(width=with_btn, x_offset=x_offset_btn, y_offset=y_offset_btn(2), url=f'internal:{int(lesson)}!A{total_row}')
                        worksheet.insert_textbox(row_index, col_index, f"CUỐI TRANG", bottom_sheet_options)
                        # ====================================================================
                        home_condition = lesson != f'{first_lesson}'
                        lesson_back_id = f"Bài {int(lesson) - 1}" if home_condition else f"HOME"
                        back_url = f'internal:{int(lesson) - 1}!A1' if home_condition else f'internal:{home_sheet_name}!A1'
                        options = button_options(width=with_btn, x_offset=x_offset_btn, y_offset=y_offset_btn(3), url=back_url)
                        worksheet.insert_textbox(row_index, col_index, lesson_back_id, options)
                        # ====================================================================
                        first_condition = lesson != f'{first_lesson + total_lesson - 1}'
                        lesson_go_id = f"Bài {int(lesson) + 1}" if first_condition else f"HOME"
                        netx_url = f'internal:{int(lesson) + 1}!A1' if first_condition else f'internal:{home_sheet_name}!A1'
                        options = button_options(width=with_btn, x_offset=x_offset_btn, y_offset=y_offset_btn(4), url=netx_url)
                        worksheet.insert_textbox(row_index, col_index, lesson_go_id, options)
                        # ====================================================================

            row_index = total_row  
            col_index = 4
            with_btn = 155
            height_btn_right = 35
            x_offset_btn =  lambda index: index*(with_btn + 20) + 20
            y_offset_btn = 20
            # ====================================================================
            home_sheet_options = button_options(width=with_btn, x_offset=x_offset_btn(0), y_offset=y_offset_btn, url=f'internal:{home_sheet_name}!A1')
            worksheet.insert_textbox(row_index, col_index, home_sheet_name, home_sheet_options)
            # ====================================================================
            col_word = lambda col: chr(65 + col)
            col_index_goto = len(word_list)+1 if len(word_list) <= 12 else int(len(word_list) / 2)+1
            options = button_options(width=with_btn, x_offset=x_offset_btn(1), y_offset=y_offset_btn, url=f'internal:{all_sheet_name}!{col_word(col_index_goto)}{int(lesson)+1}')
            worksheet.insert_textbox(row_index, col_index, 'ALL TAB', options)
            # ====================================================================
            top_sheet_options = button_options(width=with_btn, x_offset=x_offset_btn(2), y_offset=y_offset_btn, url=f'internal:{int(lesson)}!A1')
            worksheet.insert_textbox(row_index, col_index, f"ĐẦU TRANG", top_sheet_options)
            # ====================================================================
            home_condition = lesson != f'{first_lesson}'
            lesson_back_id = f"Bài {int(lesson) - 1}" if home_condition else f"HOME"
            back_url = f'internal:{int(lesson) - 1}!A1' if home_condition else f'internal:{home_sheet_name}!A1'
            options = button_options(width=with_btn, x_offset=x_offset_btn(3), y_offset=y_offset_btn, url=back_url)
            worksheet.insert_textbox(row_index, col_index, lesson_back_id, options)
            # ====================================================================
            first_condition = lesson != f'{first_lesson + total_lesson - 1}'
            lesson_go_id = f"Bài {int(lesson) + 1}" if first_condition else f"HOME"
            netx_url = f'internal:{int(lesson) + 1}!A1' if first_condition else f'internal:{home_sheet_name}!A1'
            options = button_options(width=with_btn, x_offset=x_offset_btn(4), y_offset=y_offset_btn, url=netx_url)
            worksheet.insert_textbox(row_index, col_index, lesson_go_id, options)
            # ====================================================================

    workbook.close()
print(f">> Create OKE!!")

database

>> Course:  N5
>> Open: [ Output\N5 Kanji ByVing.xlsx ]
>> Create sheet: HOME
>> Create sheet: ALL

>> Course:  N4
>> Open: [ Output\N4 Kanji ByVing.xlsx ]
>> Create sheet: HOME
>> Create sheet: ALL

>> Course:  Course
>> Open: [ Output\Course Kanji ByVing.xlsx ]
>> Create sheet: HOME
>> Create sheet: ALL
>> Create OKE!!
